# Additional Cython Features

## Automatic Type Inference Using Cython's `infer_types`

In [ ]:
import numpy as np
from random import random
import Cython

%load_ext Cython

In [ ]:
%%cython -a

from random import random
from cython cimport infer_types

cdef inline double my_rand():
    return random()

@infer_types(True)
cpdef pi_mc_inferred(n=1000):
    '''Calculate PI using Monte Carlo method'''
    in_circle = 0
    for i in range(n):
        x = my_rand()
        y = my_rand()
        if x * x + y * y <= 1.0:
            in_circle += 1
        
    return 4.0 * in_circle / n

In [ ]:
%time pi_mc_inferred(10000000)

## Cython Extensions Types

In [ ]:
class PyRectangle:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def area(self):
        return self.x * self.y
    
    def perimeter(self):
        return 2.0 * (self.x + self.y)

In [ ]:
%%cython 

cdef class CyRectangle:
    cdef:
        double x, y
        
    def __cinit__(self, x, y):
        self.x = x
        self.y = y
        
    cpdef double area(self):
        return self.x * self.y
    
    cpdef double perimeter(self):
        return 2.0 * (self.x + self.y)

In [ ]:
a = CyRectangle(1, 2)
print(a.area(), a.perimeter())

In [ ]:
%%cython
from random import random

cdef class CyRectangle:
    cdef:
        double x, y
        
    def __cinit__(self, x, y):
        self.x = x
        self.y = y
        
    cpdef double area(self):
        return self.x * self.y
    
    cpdef double perimeter(self):
        return 2.0 * (self.x + self.y)
    
cdef class CyRectangles:
    cdef:
        list rectangles
        
    def __cinit__(self, int n):
        cdef unsigned int i
        self.rectangles = []
        for i in range(n):
            self.rectangles.append(CyRectangle(random(), random()))
            
    cpdef double total_area(self):
        cdef CyRectangle rect
        cdef double area = 0.0
        for rect in self.rectangles:
            area += rect.area()
            
        return area

In [ ]:
a = CyRectangles(100000)

In [ ]:
a.total_area()

## C-like Allocation/Dealllocation

In [ ]:
%%cython

from libc.stdlib cimport malloc, free


cdef class CyRangeVector:
    cdef:
        int *data
        int size
        
    def __cinit__(self, int start, int end):
        cdef unsigned int i
        if start >= end:
            raise Exception(f'{start} >= {end}')
        self.size = end - start
        self.data = <int*>malloc(self.size * sizeof(int))
        
        for i in range(start, end):
            self.data[i - start] = i
            
    def __getitem__(self, int i):
        if i >= self.size or i < 0:
            return -1

        return self.data[i]
    
    def __dealloc__(self):
        free(self.data)

In [ ]:
my_range = CyRangeVector(10, 11000)
my_range[2]

## Interacting with the C++ Standard Template Library

As long as we start using the C++ STL from inside Cython we have to switch to `language=c++`

In [ ]:
%%cython

# distutils: language=c++

from libcpp.vector cimport vector

cdef class CyRangeVector:
    cdef:
        vector[int] data
        
    def __cinit__(self, int start, int end):
        cdef unsigned int i
        if start >= end:
            raise Exception(f'{start} >= {end}')
        for i in range(start, end):
            self.data.push_back(i)
            
    def __getitem__(self, int i):
        if i >= self.data.size() or i < 0:
            return None
        
        return self.data[i]

In [ ]:
v = CyRangeVector(1, 20)
print(v[1])

In [ ]:
%%cython

# distutils: language=c++

from libcpp.vector cimport vector

cpdef vector[int] cy_range(int start, int end):
    cdef vector[int] v
    cdef unsigned int i
    for i in range(start, end):
        v.push_back(i)
    
    return v

In [ ]:
x = cy_range(1, 10)
print(x, type(x))

### Additional STL libraries are available and you can look at their [declaration files](https://github.com/cython/cython/tree/master/Cython/Includes/libcpp)